In [87]:
import pandas as pd
from datetime import datetime
from datetime import date
import numpy as np

In [88]:
df_sales = pd.read_csv('../data/project/NYC Residential Sales Master.csv', index_col=0)

In [11]:
df_sales.head()

,ADDRESS,BOROUGH,NEIGHBORHOOD,TAX CLASS,BLOCK,LOT,ZIP,RESI UNITS,COMM UNITS,# UNITS,...,LATITUDE,bank,bus_station,gym,hospital,park,restaurant,school,shopping_mall,subway_station
2,2186 BATHGATE AVENUE,2,BATHGATE,1,3048,19,10457,1,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4443 PARK AVENUE,2,BATHGATE,1,3030,71,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4435 PARK AVENUE,2,BATHGATE,1,3030,76,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,459 EAST 179 STREET,2,BATHGATE,1,3036,62,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,459 EAST 179 STREET,2,BATHGATE,1,3036,62,10457,2,0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
df_sales['SALE DATE'] = df_sales['SALE DATE'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))

In [81]:
def category_date(d):
    if date(2009,1,1) <= d <= date(2010,12,31):
        return 'p1'
    if date(2011,1,1) <= d <= date(2012,12,31):
        return 'p2'
    if date(2013,1,1) <= d <= date(2014,12,31):
        return 'p3'
    if date(2015,1,1) <= d <= date(2016,12,31):
        return 'p4'
    if date(2017,1,1) <= d <= date(2018,12,31):
        return 'p5'
    if d >= date(2019,1,1):
        return 'p6'
    return None

dict_p = {
    'p1': '2009-2010',
    'p2': '2011-2012',
    'p3': '2013-2014',
    'p4': '2015-2016',
    'p5': '2017-2018',
    'p6': '2019'
}

In [53]:
df_sales['period'] = df_sales['SALE DATE'].apply(category_date)

In [46]:
groups = [p for p in df_sales['period'].unique() if p is not None]

In [69]:
df_sales_p = df_sales[df_sales['period']=='p1']
df_sales_p = df_sales_p.drop_duplicates(keep='first')\
                       .reset_index(drop=True)\
                       .groupby(['ZIP'])\
                       .agg({'SALE PRICE': ['mean', 'median', 'count']})\
                       .drop(0)

df_sales_p.columns = [f"average sales price {dict_p['p1']}", 
                      f"median sales price {dict_p['p1']}", 
                      f"number of transactions {dict_p['p1']}"]
df_sales_p.reset_index()\
          .astype(dtype={'ZIP': str})\
          .reset_index(drop=True)

,ZIP,average sales price 2009-2010,median sales price 2009-2010,number of transactions 2009-2010
0,10001,3.716200e+06,4150000.0,5
1,10002,2.982263e+06,2600000.0,19
2,10003,5.065943e+06,4300000.0,45
3,10007,9.451570e+06,9451570.0,1
4,10009,2.654923e+06,1450000.0,24
...,...,...,...,...
166,11436,2.899352e+05,283000.0,546
167,11691,3.470800e+05,311552.0,536
168,11692,4.035145e+05,330000.0,211
169,11693,3.087198e+05,299250.0,106


In [82]:
def group_sales(period, df):
    df_sales_p = df[df['period']==period]
    df_sales_p = df.drop_duplicates(keep='first')\
                   .reset_index(drop=True)\
                   .groupby(['ZIP'])\
                   .agg({'SALE PRICE': ['mean', 
                                        'median', 
                                        'count']})

    df_sales_p.columns = [f"average sales price {dict_p[period]}", 
                          f"median sales price {dict_p[period]}", 
                          f"number of transactions {dict_p[period]}"]
    
    df_sales_p = df_sales_p.reset_index()\
                           .astype(dtype={'ZIP': str})\
                           .reset_index(drop=True)
    
    return df_sales_p

In [73]:
group_sales('p1', df_sales)

,ZIP,average sales price 2009-2010,median sales price 2009-2010,number of transactions 2009-2010
0,10001,3.716200e+06,4150000.0,5
1,10002,2.982263e+06,2600000.0,19
2,10003,5.065943e+06,4300000.0,45
3,10007,9.451570e+06,9451570.0,1
4,10009,2.654923e+06,1450000.0,24
...,...,...,...,...
166,11436,2.899352e+05,283000.0,546
167,11691,3.470800e+05,311552.0,536
168,11692,4.035145e+05,330000.0,211
169,11693,3.087198e+05,299250.0,106


In [89]:
def gen_sales_summary(df_sales, export_path):
    df_sales['SALE DATE'] = df_sales['SALE DATE'].apply(lambda x: datetime.strptime(x, '%m/%d/%Y'))
    df_sales['period'] = df_sales['SALE DATE'].apply(category_date)
    
    groups = [p for p in df_sales['period'].unique() if p is not None]
    
    for period in groups:
        df_sales_p = group_sales(period, df_sales)
        df_sales_p.to_csv(f'{export_path}/sales_summary_{period}.csv', index=False)

In [90]:
gen_sales_summary(df_sales, '../data/project/sales_summary')